<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/00_ExtraerCaracteristicas_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
SISTEMA DE EXTRACCIÓN DE CARACTERÍSTICAS FOTOGRÁFICAS
================================================================================

Sistema profesional de extracción exhaustiva de características de imágenes JPEG
derivadas de archivos RAW para análisis comparativo de técnicas de segmentación.

OBJETIVO:
Extraer características técnicas y visuales completas de fotografías JPEG para
análisis de calidad, textura, color, nitidez, exposición y saliencia en el
contexto de segmentación de personas en fotografía de retrato.

CARACTERÍSTICAS EXTRAÍDAS:
1. Metadatos completos de archivo y EXIF
2. Estadísticas de color en RGB, HSV y LAB
3. Histogramas detallados (RGB + intensidad) con entropía
4. Texturas avanzadas: Haralick, GLCM, LBP, Zernike
5. Bordes y características geométricas: Canny multi-sigma, esquinas, gradientes, HOG
6. Análisis de calidad: nitidez, exposición, contraste, balance de blancos, ruido
7. Análisis de frecuencias: energía por bandas, entropía espectral
8. Saliencia visual: espectral, distribución espacial, centroide
9. Colores dominantes con paleta y frecuencias
10. Hashes perceptuales completos

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña (UOC)
Fecha: Noviembre 2025
================================================================================
"""

'\n================================================================================\nSISTEMA DE EXTRACCIÓN DE CARACTERÍSTICAS FOTOGRÁFICAS\n================================================================================\n\nSistema profesional de extracción exhaustiva de características de imágenes JPEG\nderivadas de archivos RAW para análisis comparativo de técnicas de segmentación.\n\nOBJETIVO:\nExtraer características técnicas y visuales completas de fotografías JPEG para\nanálisis de calidad, textura, color, nitidez, exposición y saliencia en el\ncontexto de segmentación de personas en fotografía de retrato.\n\nCARACTERÍSTICAS EXTRAÍDAS:\n1. Metadatos completos de archivo y EXIF\n2. Estadísticas de color en RGB, HSV y LAB\n3. Histogramas detallados (RGB + intensidad) con entropía\n4. Texturas avanzadas: Haralick, GLCM, LBP, Zernike\n5. Bordes y características geométricas: Canny multi-sigma, esquinas, gradientes, HOG\n6. Análisis de calidad: nitidez, exposición, contraste, balance 

In [ ]:
# =============================================================================
# GESTIÓN DE DEPENDENCIAS
# =============================================================================

class GestorDependencias:
    """Gestiona la instalación automática de dependencias necesarias."""

    DEPENDENCIAS_REQUERIDAS = [
        'piexif',
        'exifread',
        'imagehash',
        'scikit-image',
        'mahotas',
        'opencv-python',
        'tqdm',
    ]

    @classmethod
    def instalar_todas(cls) -> bool:
        """Instala todas las dependencias requeridas."""
        print("=" * 80)
        print("INSTALANDO DEPENDENCIAS")
        print("=" * 80)

        exitosas = 0
        fallidas = 0

        for dependencia in cls.DEPENDENCIAS_REQUERIDAS:
            print(f"\nInstalando {dependencia}...", end=" ")
            try:
                subprocess.check_call(
                    [sys.executable, '-m', 'pip', 'install', '-q', dependencia],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL
                )
                print("OK")
                exitosas += 1
            except subprocess.CalledProcessError as e:
                print(f"ERROR: {str(e)}")
                fallidas += 1

        print(f"\nResumen: {exitosas} exitosas, {fallidas} fallidas")
        return fallidas == 0


def montar_google_drive() -> bool:
    """Monta Google Drive si se ejecuta en Google Colab."""
    try:
        from google.colab import drive
        print("Montando Google Drive...", end=" ")
        drive.mount('/content/drive', force_remount=False)
        print("OK\n")
        return True
    except ImportError:
        print("Ejecutando fuera de Google Colab. Sin montaje de Drive.\n")
        return True
    except Exception as e:
        print(f"Advertencia al montar Drive: {str(e)}\n")
        return False

In [ ]:
import subprocess
import sys
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass
import json
import logging
import warnings
import time

import numpy as np
import cv2
from PIL import Image
import piexif
import exifread
import imagehash
from tqdm import tqdm

from skimage import color, feature, filters, measure, morphology, exposure
from scipy import ndimage, stats
from sklearn.cluster import KMeans
import mahotas as mh

warnings.filterwarnings('ignore')

In [ ]:
# =============================================================================
# CONFIGURACIÓN DEL SISTEMA
# =============================================================================

@dataclass
class ConfiguracionExtraccion:
    """Configuración centralizada del sistema de extracción."""

    ruta_base: Path = Path("/content/drive/MyDrive/TFM/")
    ruta_imagenes: Optional[Path] = None
    ruta_resultados: Optional[Path] = None
    max_dimension_procesamiento: int = 2048

    calcular_exif: bool = True
    calcular_histogramas: bool = True
    calcular_texturas: bool = True
    calcular_bordes: bool = True
    calcular_frecuencias: bool = True
    calcular_calidad: bool = True
    calcular_saliencia: bool = True
    calcular_colores_dominantes: bool = True
    calcular_hashes: bool = True

    num_colores_paleta: int = 5
    guardar_json_individual: bool = True
    guardar_json_consolidado: bool = True
    nivel_log: str = "INFO"

    def __post_init__(self):
        """Inicializa rutas derivadas si no fueron especificadas."""
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "1_Caracteristicas"

    def crear_estructura_directorios(self) -> None:
        """Crea la estructura de directorios necesaria para resultados."""
        self.ruta_resultados.mkdir(parents=True, exist_ok=True)
        (self.ruta_resultados / "json").mkdir(exist_ok=True)
        (self.ruta_resultados / "logs").mkdir(exist_ok=True)

    def validar(self) -> Tuple[bool, Optional[str]]:
        """Valida la configuración antes de procesar."""
        if not self.ruta_imagenes.exists():
            return False, f"Directorio de imágenes no existe: {self.ruta_imagenes}"

        if self.max_dimension_procesamiento < 256:
            return False, "max_dimension_procesamiento debe ser >= 256"

        if not (1 <= self.num_colores_paleta <= 20):
            return False, "num_colores_paleta debe estar entre 1 y 20"

        return True, None

In [ ]:
# =============================================================================
# SISTEMA DE LOGGING
# =============================================================================

class LoggerManager:
    """Gestor centralizado del sistema de logging con trazabilidad completa."""

    def __init__(self, directorio_logs: Path, nivel: str = "INFO"):
        """Inicializa el sistema de logging."""
        self.directorio_logs = directorio_logs
        self.directorio_logs.mkdir(parents=True, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archivo_log = self.directorio_logs / f"extraccion_{timestamp}.log"

        formato = '%(asctime)s | %(name)-25s | %(levelname)-8s | %(message)s'

        logging.basicConfig(
            level=getattr(logging, nivel.upper()),
            format=formato,
            handlers=[
                logging.FileHandler(archivo_log, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )

        self.logger = logging.getLogger('ExtractorCaracteristicas')
        self.logger.info(f"Sistema de logging inicializado: {archivo_log.name}")

    def get_logger(self) -> logging.Logger:
        """Retorna el logger configurado."""
        return self.logger

In [ ]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones de utilidad para conversión y serialización de datos."""

    @staticmethod
    def convertir_a_serializable(obj: Any) -> Any:
        """Convierte objetos a tipos serializables en JSON."""
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, bytes):
            return obj.decode('utf-8', errors='ignore')
        elif isinstance(obj, (tuple, list)):
            return [Utilidades.convertir_a_serializable(item) for item in obj]
        elif isinstance(obj, dict):
            return {k: Utilidades.convertir_a_serializable(v) for k, v in obj.items()}
        return obj

In [ ]:
# =============================================================================
# CARGADOR DE IMÁGENES
# =============================================================================

class CargadorImagen:
    """Carga y valida imágenes JPEG con manejo robusto de errores."""

    EXTENSIONES_VALIDAS = {'.jpg', '.jpeg'}

    def __init__(self, config: ConfiguracionExtraccion):
        """Inicializa el cargador."""
        self.config = config
        self.logger = logging.getLogger('CargadorImagen')

    def cargar_imagen(self, ruta_imagen: Path) -> Tuple[Optional[np.ndarray], Dict[str, Any]]:
        """Carga una imagen JPEG desde archivo."""
        if not ruta_imagen.exists():
            self.logger.error(f"Archivo no existe: {ruta_imagen}")
            return None, {}

        try:
            img_pil = Image.open(ruta_imagen)

            if img_pil.mode != 'RGB':
                img_pil = img_pil.convert('RGB')

            img_rgb = np.array(img_pil)

            stat_info = ruta_imagen.stat()

            metadatos = {
                'formato_original': img_pil.format,
                'modo_color': img_pil.mode,
                'ancho_original': img_pil.width,
                'alto_original': img_pil.height,
                'tamano_archivo_bytes': stat_info.st_size,
                'tamano_archivo_mb': round(stat_info.st_size / (1024 * 1024), 3),
                'fecha_modificacion': datetime.fromtimestamp(stat_info.st_mtime).isoformat()
            }

            self.logger.debug(f"Imagen cargada: {ruta_imagen.name} | "
                            f"Dimensiones: {img_rgb.shape} | "
                            f"Tamaño: {metadatos['tamano_archivo_mb']} MB")

            return img_rgb, metadatos

        except Exception as e:
            self.logger.error(f"Error cargando imagen {ruta_imagen.name}: {str(e)}")
            return None, {}

In [ ]:
# =============================================================================
# EXTRACTORES DE CARACTERÍSTICAS
# =============================================================================

class ExtractorEXIF:
    """Extrae metadatos EXIF completos de archivos JPEG."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def extraer_exif(self, ruta_imagen: Path) -> Dict[str, Any]:
        """Extrae metadatos EXIF completos del archivo."""
        exif_data = {}

        try:
            with open(ruta_imagen, 'rb') as f:
                tags = exifread.process_file(f, details=False)

            mapeo_tags = {
                'Image Make': 'fabricante_camara',
                'Image Model': 'modelo_camara',
                'EXIF LensModel': 'modelo_lente',
                'EXIF FocalLength': 'distancia_focal',
                'EXIF FNumber': 'apertura',
                'EXIF ExposureTime': 'tiempo_exposicion',
                'EXIF ISOSpeedRatings': 'iso',
                'EXIF DateTimeOriginal': 'fecha_captura',
                'EXIF Flash': 'flash',
                'EXIF WhiteBalance': 'balance_blancos',
                'EXIF ExposureProgram': 'programa_exposicion',
                'EXIF MeteringMode': 'modo_medicion',
                'GPS GPSLatitude': 'latitud',
                'GPS GPSLongitude': 'longitud',
                'Image Orientation': 'orientacion',
                'EXIF ColorSpace': 'espacio_color',
            }

            for tag_exif, nombre_campo in mapeo_tags.items():
                if tag_exif in tags:
                    exif_data[nombre_campo] = str(tags[tag_exif])

            if 'tiempo_exposicion' in exif_data and '/' in exif_data['tiempo_exposicion']:
                try:
                    num, den = exif_data['tiempo_exposicion'].split('/')
                    exif_data['tiempo_exposicion_segundos'] = float(num) / float(den)
                except Exception:
                    pass

            if 'apertura' in exif_data and '/' in exif_data['apertura']:
                try:
                    num, den = exif_data['apertura'].split('/')
                    exif_data['apertura_fnumber'] = float(num) / float(den)
                except Exception:
                    pass

            self.logger.debug(f"EXIF extraído: {len(exif_data)} campos")

        except Exception as e:
            self.logger.warning(f"Error extrayendo EXIF: {str(e)}")

        return exif_data


class AnalizadorColor:
    """Análisis completo de características de color."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_color_completo(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo de color en múltiples espacios."""
        resultado = {}

        try:
            resultado['rgb'] = self._estadisticas_rgb(img_rgb)

            img_hsv = color.rgb2hsv(img_rgb / 255.0)
            resultado['hsv'] = self._estadisticas_hsv(img_hsv)

            img_lab = color.rgb2lab(img_rgb)
            resultado['lab'] = self._estadisticas_lab(img_lab)

            resultado['global'] = self._estadisticas_globales(img_rgb)

            self.logger.debug("Análisis de color completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de color: {str(e)}")

        return resultado

    def _estadisticas_rgb(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Calcula estadísticas completas en espacio RGB."""
        stats = {
            'mean': [],
            'median': [],
            'stddev': [],
            'variance': []
        }

        for i in range(3):
            datos_canal = img_rgb[:, :, i]
            stats['mean'].append(round(float(np.mean(datos_canal)), 2))
            stats['median'].append(round(float(np.median(datos_canal)), 2))
            stats['stddev'].append(round(float(np.std(datos_canal)), 2))
            stats['variance'].append(round(float(np.var(datos_canal)), 2))

        return stats

    def _estadisticas_hsv(self, img_hsv: np.ndarray) -> Dict[str, Any]:
        """Calcula estadísticas en espacio HSV."""
        return {
            'hue_mean': round(float(np.mean(img_hsv[:, :, 0])), 4),
            'hue_std': round(float(np.std(img_hsv[:, :, 0])), 4),
            'saturation_mean': round(float(np.mean(img_hsv[:, :, 1])), 4),
            'saturation_std': round(float(np.std(img_hsv[:, :, 1])), 4),
            'value_mean': round(float(np.mean(img_hsv[:, :, 2])), 4),
            'value_std': round(float(np.std(img_hsv[:, :, 2])), 4)
        }

    def _estadisticas_lab(self, img_lab: np.ndarray) -> Dict[str, Any]:
        """Calcula estadísticas en espacio LAB."""
        return {
            'l_mean': round(float(np.mean(img_lab[:, :, 0])), 2),
            'l_std': round(float(np.std(img_lab[:, :, 0])), 2),
            'a_mean': round(float(np.mean(img_lab[:, :, 1])), 2),
            'a_std': round(float(np.std(img_lab[:, :, 1])), 2),
            'b_mean': round(float(np.mean(img_lab[:, :, 2])), 2),
            'b_std': round(float(np.std(img_lab[:, :, 2])), 2)
        }

    def _estadisticas_globales(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Calcula estadísticas globales de la imagen."""
        img_gray = color.rgb2gray(img_rgb)

        brillo_promedio = float(np.mean(img_gray) * 255)
        contraste = float(np.std(img_gray) * 255)

        img_gray_uint8 = (img_gray * 255).astype(np.uint8)
        hist, _ = np.histogram(img_gray_uint8, bins=256, range=(0, 256))
        hist_norm = hist / hist.sum()
        entropia = -np.sum(hist_norm[hist_norm > 0] * np.log2(hist_norm[hist_norm > 0]))

        return {
            'brillo_promedio': round(brillo_promedio, 2),
            'contraste': round(contraste, 2),
            'entropia': round(float(entropia), 4)
        }


class AnalizadorHistogramas:
    """Análisis detallado de histogramas RGB e intensidad."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_histogramas(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Calcula histogramas completos con estadísticas."""
        resultado = {}

        try:
            canales_nombres = ['rojo', 'verde', 'azul']

            for i, nombre in enumerate(canales_nombres):
                hist, _ = np.histogram(img_rgb[:, :, i], bins=256, range=(0, 256))
                pico_principal = int(np.argmax(hist))

                resultado[nombre] = {
                    'histograma': hist.tolist(),
                    'pico_principal': pico_principal,
                    'media': round(float(np.mean(img_rgb[:, :, i])), 2),
                    'std': round(float(np.std(img_rgb[:, :, i])), 2)
                }

            img_gray = color.rgb2gray(img_rgb)
            img_gray_uint8 = (img_gray * 255).astype(np.uint8)
            hist_intensidad, _ = np.histogram(img_gray_uint8, bins=256, range=(0, 256))

            hist_norm = hist_intensidad / hist_intensidad.sum()
            entropia = -np.sum(hist_norm[hist_norm > 0] * np.log2(hist_norm[hist_norm > 0]))

            resultado['intensidad'] = {
                'histograma': hist_intensidad.tolist(),
                'entropia': round(float(entropia), 4)
            }

            self.logger.debug("Histogramas calculados")

        except Exception as e:
            self.logger.warning(f"Error calculando histogramas: {str(e)}")

        return resultado


class AnalizadorTextura:
    """Análisis exhaustivo de características de textura."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_textura_completa(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo de texturas: Haralick, GLCM, LBP, Zernike."""
        resultado = {}

        try:
            img_gray = color.rgb2gray(img_rgb)
            img_gray_uint8 = (img_gray * 255).astype(np.uint8)

            resultado['haralick'] = self._calcular_haralick(img_gray_uint8)
            resultado['glcm'] = self._calcular_glcm(img_gray_uint8)
            resultado['lbp'] = self._calcular_lbp(img_gray_uint8)
            resultado['zernike'] = self._calcular_zernike(img_gray_uint8)

            self.logger.debug("Análisis de textura completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de textura: {str(e)}")

        return resultado

    def _calcular_haralick(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula características de Haralick."""
        try:
            haralick_4dir = mh.features.haralick(img_gray)
            haralick_mean = haralick_4dir.mean(axis=0)

            nombres = [
                'angular_second_moment',
                'contrast',
                'correlation',
                'sum_of_squares',
                'inverse_diff_moment',
                'sum_average',
                'sum_variance',
                'sum_entropy',
                'entropy',
                'difference_variance',
                'difference_entropy',
                'info_measure_correlation_1',
                'info_measure_correlation_2'
            ]

            resultado = {}
            for i, nombre in enumerate(nombres):
                if i < len(haralick_mean):
                    resultado[nombre] = round(float(haralick_mean[i]), 6)

            return resultado
        except Exception as e:
            self.logger.warning(f"Error calculando Haralick: {str(e)}")
            return {}

    def _calcular_glcm(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula características GLCM (Gray-Level Co-occurrence Matrix)."""
        try:
            distancias = [1, 2, 3]
            angulos = [0, np.pi/4, np.pi/2, 3*np.pi/4]

            glcm = feature.graycomatrix(
                img_gray,
                distances=distancias,
                angles=angulos,
                levels=256,
                symmetric=True,
                normed=True
            )

            propiedades = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
            resultado = {}

            for prop in propiedades:
                valores = feature.graycoprops(glcm, prop).flatten()
                resultado[f'{prop}_mean'] = round(float(np.mean(valores)), 6)
                resultado[f'{prop}_std'] = round(float(np.std(valores)), 6)

            return resultado
        except Exception as e:
            self.logger.warning(f"Error calculando GLCM: {str(e)}")
            return {}

    def _calcular_lbp(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula características LBP (Local Binary Patterns)."""
        try:
            radius = 3
            n_points = 8 * radius

            lbp = feature.local_binary_pattern(img_gray, n_points, radius, method='uniform')

            hist, _ = np.histogram(lbp.ravel(), bins=n_points + 2, range=(0, n_points + 2))
            hist_norm = hist / hist.sum()
            entropia = -np.sum(hist_norm[hist_norm > 0] * np.log2(hist_norm[hist_norm > 0]))

            return {
                'mean': round(float(np.mean(lbp)), 4),
                'std': round(float(np.std(lbp)), 2),
                'entropy': round(float(entropia), 6)
            }
        except Exception as e:
            self.logger.warning(f"Error calculando LBP: {str(e)}")
            return {}

    def _calcular_zernike(self, img_gray: np.ndarray) -> List[float]:
        """Calcula momentos de Zernike."""
        try:
            radius = min(img_gray.shape) // 2
            zernike_moments = mh.features.zernike_moments(img_gray, radius, degree=8)
            return [round(float(z), 6) for z in zernike_moments]
        except Exception as e:
            self.logger.warning(f"Error calculando Zernike: {str(e)}")
            return []


class AnalizadorBordes:
    """Análisis completo de bordes y características geométricas."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_bordes_completo(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo: Canny, esquinas, gradientes, HOG."""
        resultado = {}

        try:
            img_gray = color.rgb2gray(img_rgb)

            resultado['canny'] = self._analizar_canny_multisigma(img_gray)
            resultado['esquinas'] = self._detectar_esquinas(img_gray)
            resultado['gradientes'] = self._analizar_gradientes(img_gray)
            resultado['hog'] = self._calcular_hog(img_gray)

            self.logger.debug("Análisis de bordes completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de bordes: {str(e)}")

        return resultado

    def _analizar_canny_multisigma(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Detecta bordes con Canny usando múltiples sigmas."""
        resultado = {}

        for sigma in [1.0, 2.0, 3.0]:
            try:
                bordes = feature.canny(img_gray, sigma=sigma)
                densidad = float(np.sum(bordes) / bordes.size)
                resultado[f'densidad_sigma_{sigma}'] = round(densidad, 6)
            except Exception:
                pass

        return resultado

    def _detectar_esquinas(self, img_gray: np.ndarray) -> Dict[str, Any]:
        """Detecta esquinas usando Harris y Shi-Tomasi."""
        try:
            img_gray_uint8 = (img_gray * 255).astype(np.uint8)

            harris = cv2.cornerHarris(img_gray_uint8, blockSize=2, ksize=3, k=0.04)
            num_harris = int(np.sum(harris > 0.01 * harris.max()))

            shi_tomasi = cv2.goodFeaturesToTrack(img_gray_uint8, maxCorners=20000,
                                                  qualityLevel=0.01, minDistance=10)
            num_shi_tomasi = len(shi_tomasi) if shi_tomasi is not None else 0

            return {
                'harris': num_harris,
                'shi_tomasi': num_shi_tomasi,
                'densidad_harris': round(num_harris / img_gray.size, 6)
            }
        except Exception as e:
            self.logger.warning(f"Error detectando esquinas: {str(e)}")
            return {}

    def _analizar_gradientes(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Analiza gradientes usando varios operadores."""
        try:
            sobel = filters.sobel(img_gray)
            scharr = filters.scharr(img_gray)
            prewitt = filters.prewitt(img_gray)

            return {
                'sobel_mean': round(float(np.mean(sobel)), 6),
                'sobel_max': round(float(np.max(sobel)), 6),
                'sobel_std': round(float(np.std(sobel)), 6),
                'scharr_mean': round(float(np.mean(scharr)), 6),
                'prewitt_mean': round(float(np.mean(prewitt)), 6)
            }
        except Exception as e:
            self.logger.warning(f"Error analizando gradientes: {str(e)}")
            return {}

    def _calcular_hog(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula HOG (Histogram of Oriented Gradients)."""
        try:
            hog_features = feature.hog(img_gray, orientations=9, pixels_per_cell=(8, 8),
                                       cells_per_block=(2, 2), feature_vector=True)

            return {
                'mean': round(float(np.mean(hog_features)), 6),
                'std': round(float(np.std(hog_features)), 6),
                'max': round(float(np.max(hog_features)), 6)
            }
        except Exception as e:
            self.logger.warning(f"Error calculando HOG: {str(e)}")
            return {}


class AnalizadorCalidad:
    """Análisis completo de calidad fotográfica."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_calidad_completa(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo: nitidez, exposición, contraste, balance de blancos, ruido."""
        resultado = {}

        try:
            img_gray = color.rgb2gray(img_rgb)

            resultado['nitidez'] = self._analizar_nitidez(img_gray)
            resultado['exposicion'] = self._analizar_exposicion(img_gray)
            resultado['contraste'] = self._analizar_contraste(img_gray)
            resultado['balance_blancos'] = self._analizar_balance_blancos(img_rgb)
            resultado['ruido'] = self._estimar_ruido(img_gray)

            self.logger.debug("Análisis de calidad completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de calidad: {str(e)}")

        return resultado

    def _analizar_nitidez(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Analiza nitidez usando varios métodos."""
        try:
            laplacian = cv2.Laplacian((img_gray * 255).astype(np.uint8), cv2.CV_64F)
            laplacian_var = float(np.var(laplacian))

            gx = ndimage.sobel(img_gray, axis=0)
            gy = ndimage.sobel(img_gray, axis=1)
            tenengrad = float(np.mean(gx**2 + gy**2))

            varianza_normalizada = float(np.std(img_gray) / np.mean(img_gray) * 100)

            return {
                'laplacian': round(laplacian_var, 4),
                'tenengrad': round(tenengrad, 4),
                'varianza_normalizada': round(varianza_normalizada, 4)
            }
        except Exception as e:
            self.logger.warning(f"Error analizando nitidez: {str(e)}")
            return {}

    def _analizar_exposicion(self, img_gray: np.ndarray) -> Dict[str, Any]:
        """Analiza exposición y rango dinámico."""
        try:
            img_gray_uint8 = (img_gray * 255).astype(np.uint8)

            brillo_medio = float(np.mean(img_gray_uint8))
            sobre_expuesto = float(np.sum(img_gray_uint8 >= 250) / img_gray_uint8.size * 100)
            sub_expuesto = float(np.sum(img_gray_uint8 <= 5) / img_gray_uint8.size * 100)

            hist, _ = np.histogram(img_gray_uint8, bins=256, range=(0, 256))
            hist_norm = hist / hist.sum()
            entropia = -np.sum(hist_norm[hist_norm > 0] * np.log2(hist_norm[hist_norm > 0]))

            rango_dinamico = int(np.max(img_gray_uint8) - np.min(img_gray_uint8))

            return {
                'brillo_medio': round(brillo_medio, 2),
                'sobre_expuesto_pct': round(sobre_expuesto, 3),
                'sub_expuesto_pct': round(sub_expuesto, 3),
                'entropia': round(float(entropia), 4),
                'rango_dinamico': rango_dinamico
            }
        except Exception as e:
            self.logger.warning(f"Error analizando exposición: {str(e)}")
            return {}

    def _analizar_contraste(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Analiza contraste usando RMS y Michelson."""
        try:
            rms_contrast = float(np.std(img_gray) * 255)

            I_max = float(np.max(img_gray))
            I_min = float(np.min(img_gray))
            michelson = (I_max - I_min) / (I_max + I_min) if (I_max + I_min) > 0 else 0

            return {
                'rms': round(rms_contrast, 4),
                'michelson': round(michelson, 6)
            }
        except Exception as e:
            self.logger.warning(f"Error analizando contraste: {str(e)}")
            return {}

    def _analizar_balance_blancos(self, img_rgb: np.ndarray) -> Dict[str, float]:
        """Analiza balance de blancos."""
        try:
            mean_r = float(np.mean(img_rgb[:, :, 0]))
            mean_g = float(np.mean(img_rgb[:, :, 1]))
            mean_b = float(np.mean(img_rgb[:, :, 2]))

            desviacion = float(np.std([mean_r, mean_g, mean_b]))

            return {
                'mean_red': round(mean_r, 2),
                'mean_green': round(mean_g, 2),
                'mean_blue': round(mean_b, 2),
                'desviacion_canales': round(desviacion, 4)
            }
        except Exception as e:
            self.logger.warning(f"Error analizando balance de blancos: {str(e)}")
            return {}

    def _estimar_ruido(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Estima nivel de ruido en la imagen."""
        try:
            laplacian = ndimage.laplace(img_gray)
            ruido_laplacian = float(np.std(laplacian))

            img_gray_uint8 = (img_gray * 255).astype(np.uint8)
            median_filtered = cv2.medianBlur(img_gray_uint8, 5)
            noise_estimate = float(np.mean(np.abs(img_gray_uint8.astype(float) - median_filtered.astype(float))))

            signal = float(np.mean(img_gray))
            noise = ruido_laplacian
            snr = signal / noise if noise > 0 else 0
            snr_db = 20 * np.log10(snr) if snr > 0 else 0

            return {
                'laplacian': round(ruido_laplacian, 4),
                'median_filter': round(noise_estimate, 4),
                'snr': round(snr, 4),
                'snr_db': round(snr_db, 2)
            }
        except Exception as e:
            self.logger.warning(f"Error estimando ruido: {str(e)}")
            return {}


class AnalizadorFrecuencias:
    """Análisis en dominio de frecuencias usando FFT."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_frecuencias_completo(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo de frecuencias: energía por bandas, entropía espectral."""
        resultado = {}

        try:
            img_gray = color.rgb2gray(img_rgb)

            fft = np.fft.fft2(img_gray)
            fft_shift = np.fft.fftshift(fft)
            magnitud = np.abs(fft_shift)

            h, w = magnitud.shape
            centro_y, centro_x = h // 2, w // 2

            Y, X = np.ogrid[:h, :w]
            distancias = np.sqrt((X - centro_x)**2 + (Y - centro_y)**2)

            max_dist = np.sqrt(centro_x**2 + centro_y**2)
            umbral_baja = max_dist * 0.1
            umbral_alta = max_dist * 0.5

            mask_baja = distancias <= umbral_baja
            mask_media = (distancias > umbral_baja) & (distancias <= umbral_alta)
            mask_alta = distancias > umbral_alta

            energia_baja = float(np.sum(magnitud[mask_baja]**2))
            energia_media = float(np.sum(magnitud[mask_media]**2))
            energia_alta = float(np.sum(magnitud[mask_alta]**2))
            energia_total = energia_baja + energia_media + energia_alta

            magnitud_norm = magnitud / magnitud.sum()
            entropia = -np.sum(magnitud_norm[magnitud_norm > 0] * np.log2(magnitud_norm[magnitud_norm > 0]))

            resultado = {
                'energia_frecuencia_baja': round(energia_baja, 2),
                'energia_frecuencia_media': round(energia_media, 2),
                'energia_frecuencia_alta': round(energia_alta, 2),
                'ratio_baja': round(energia_baja / energia_total, 6),
                'ratio_media': round(energia_media / energia_total, 6),
                'ratio_alta': round(energia_alta / energia_total, 6),
                'entropia_espectral': round(float(entropia), 6),
                'pico_dominante': round(float(np.max(magnitud)), 2)
            }

            self.logger.debug("Análisis de frecuencias completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de frecuencias: {str(e)}")

        return resultado


class AnalizadorSaliencia:
    """Análisis completo de saliencia visual."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_saliencia_completa(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Análisis completo: saliencia espectral, distribución espacial, centroide."""
        resultado = {}

        try:
            img_gray = color.rgb2gray(img_rgb)

            resultado['espectral'] = self._calcular_saliencia_espectral(img_gray)
            resultado['distribucion_espacial'] = self._analizar_distribucion_espacial(img_gray)
            resultado['centroide'] = self._calcular_centroide(img_gray)

            self.logger.debug("Análisis de saliencia completado")

        except Exception as e:
            self.logger.warning(f"Error en análisis de saliencia: {str(e)}")

        return resultado

    def _calcular_saliencia_espectral(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula saliencia usando análisis espectral (método de Hou)."""
        try:
            fft = np.fft.fft2(img_gray)
            amplitud = np.abs(fft)
            fase = np.angle(fft)

            log_amplitud = np.log(amplitud + 1e-10)
            residual = log_amplitud - ndimage.uniform_filter(log_amplitud, size=3)

            saliency = np.abs(np.fft.ifft2(np.exp(residual + 1j * fase)))**2
            saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-10)

            return {
                'media': round(float(np.mean(saliency)), 5),
                'std': round(float(np.std(saliency)), 5),
                'max': round(float(np.max(saliency)), 6),
                'percentil_90': round(float(np.percentile(saliency, 90)), 6),
                'percentil_95': round(float(np.percentile(saliency, 95)), 6)
            }
        except Exception as e:
            self.logger.warning(f"Error calculando saliencia espectral: {str(e)}")
            return {}

    def _analizar_distribucion_espacial(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Analiza distribución espacial de la saliencia."""
        try:
            saliency = ndimage.generic_filter(img_gray, np.var, size=15)
            saliency_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-10)

            h, w = saliency_norm.shape

            cuadrantes = {
                'superior_izquierda': saliency_norm[:h//2, :w//2],
                'superior_derecha': saliency_norm[:h//2, w//2:],
                'inferior_izquierda': saliency_norm[h//2:, :w//2],
                'inferior_derecha': saliency_norm[h//2:, w//2:]
            }

            resultado = {}
            medias_cuadrantes = []

            for nombre, cuadrante in cuadrantes.items():
                media = float(np.mean(cuadrante))
                resultado[f'saliencia_{nombre}'] = round(media, 6)
                medias_cuadrantes.append(media)

            resultado['variabilidad_cuadrantes'] = round(float(np.std(medias_cuadrantes)), 5)

            centro_h, centro_w = h // 4, w // 4
            centro = saliency_norm[centro_h:3*centro_h, centro_w:3*centro_w]
            resultado['saliencia_centro'] = round(float(np.mean(centro)), 6)

            mask_periferia = np.ones_like(saliency_norm, dtype=bool)
            mask_periferia[centro_h:3*centro_h, centro_w:3*centro_w] = False
            resultado['saliencia_periferia'] = round(float(np.mean(saliency_norm[mask_periferia])), 6)

            ratio = resultado['saliencia_centro'] / (resultado['saliencia_periferia'] + 1e-10)
            resultado['ratio_centro_periferia'] = round(ratio, 6)

            return resultado
        except Exception as e:
            self.logger.warning(f"Error analizando distribución espacial: {str(e)}")
            return {}

    def _calcular_centroide(self, img_gray: np.ndarray) -> Dict[str, float]:
        """Calcula centroide de saliencia."""
        try:
            saliency = ndimage.generic_filter(img_gray, np.var, size=15)
            umbral = np.percentile(saliency, 90)
            mask_saliente = saliency > umbral

            if np.sum(mask_saliente) == 0:
                return {}

            y, x = np.where(mask_saliente)
            centroide_x = float(np.mean(x))
            centroide_y = float(np.mean(y))

            h, w = img_gray.shape
            centroide_x_norm = centroide_x / w
            centroide_y_norm = centroide_y / h

            distancia_centro = np.sqrt((centroide_x_norm - 0.5)**2 + (centroide_y_norm - 0.5)**2)

            area_saliente_pct = float(np.sum(mask_saliente) / mask_saliente.size * 100)

            return {
                'centroide_x_normalizado': round(centroide_x_norm, 4),
                'centroide_y_normalizado': round(centroide_y_norm, 4),
                'distancia_desde_centro': round(distancia_centro, 4),
                'area_saliente_porcentaje': round(area_saliente_pct, 1)
            }
        except Exception as e:
            self.logger.warning(f"Error calculando centroide: {str(e)}")
            return {}


class ExtractorColoresDominantes:
    """Extrae paleta de colores dominantes."""

    def __init__(self, logger: logging.Logger, num_colores: int = 5):
        self.logger = logger
        self.num_colores = num_colores

    def extraer_colores_dominantes(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """Extrae colores dominantes con K-means."""
        try:
            pixels = img_rgb.reshape(-1, 3)
            kmeans = KMeans(n_clusters=self.num_colores, random_state=42, n_init=10)
            kmeans.fit(pixels)

            colores = kmeans.cluster_centers_.astype(int)
            conteos = np.bincount(kmeans.labels_)

            indices_ordenados = np.argsort(conteos)[::-1]

            paleta = []
            frecuencias = []

            for idx in indices_ordenados:
                color_rgb = colores[idx]
                paleta.append(color_rgb.tolist())
                frecuencias.append(round(float(conteos[idx] / len(pixels)), 4))

            return {
                'color_dominante': paleta[0],
                'paleta': paleta,
                'frecuencias': frecuencias
            }
        except Exception as e:
            self.logger.warning(f"Error extrayendo colores dominantes: {str(e)}")
            return {}


class CalculadorHashes:
    """Calcula hashes perceptuales de imágenes."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def calcular_hashes(self, img_pil: Image.Image) -> Dict[str, str]:
        """Calcula múltiples hashes perceptuales."""
        try:
            return {
                'average_hash': str(imagehash.average_hash(img_pil)),
                'perceptual_hash': str(imagehash.phash(img_pil)),
                'difference_hash': str(imagehash.dhash(img_pil)),
                'wavelet_hash': str(imagehash.whash(img_pil)),
                'color_hash': str(imagehash.colorhash(img_pil))
            }
        except Exception as e:
            self.logger.warning(f"Error calculando hashes: {str(e)}")
            return {}

In [ ]:
# =============================================================================
# EXTRACTOR PRINCIPAL
# =============================================================================

class ExtractorCaracteristicasImagen:
    """Sistema completo de extracción de características con trazabilidad."""

    def __init__(self, config: ConfiguracionExtraccion):
        """Inicializa el extractor."""
        self.config = config
        self.config.crear_estructura_directorios()

        logger_manager = LoggerManager(
            self.config.ruta_resultados / "logs",
            self.config.nivel_log
        )
        self.logger = logger_manager.get_logger()

        self.cargador = CargadorImagen(config)

        self.extractor_exif = ExtractorEXIF(self.logger)
        self.analizador_color = AnalizadorColor(self.logger)
        self.analizador_histogramas = AnalizadorHistogramas(self.logger)
        self.analizador_textura = AnalizadorTextura(self.logger)
        self.analizador_bordes = AnalizadorBordes(self.logger)
        self.analizador_calidad = AnalizadorCalidad(self.logger)
        self.analizador_frecuencia = AnalizadorFrecuencias(self.logger)
        self.analizador_saliencia = AnalizadorSaliencia(self.logger)
        self.extractor_colores = ExtractorColoresDominantes(self.logger, config.num_colores_paleta)
        self.calculador_hashes = CalculadorHashes(self.logger)

        self.resultados = []
        self.estadisticas_procesamiento = {
            'total': 0,
            'exitosas': 0,
            'errores': 0,
            'tiempo_total': 0.0
        }

    def procesar_imagen(self, ruta_imagen: Path) -> Optional[Dict[str, Any]]:
        """Procesa una imagen individual con tracking detallado."""
        tiempo_inicio = time.time()

        self.logger.info("=" * 80)
        self.logger.info(f"Procesando: {ruta_imagen.name}")
        self.logger.info("=" * 80)

        pasos = [
            "Cargando imagen",
            "Redimensionando",
            "Extrayendo EXIF",
            "Analizando color",
            "Calculando histogramas",
            "Analizando texturas",
            "Detectando bordes",
            "Analizando calidad",
            "Analizando frecuencias",
            "Analizando saliencia",
            "Extrayendo colores",
            "Calculando hashes",
            "Guardando resultados"
        ]

        try:
            with tqdm(total=len(pasos), desc=f"  {ruta_imagen.name}",
                     bar_format='{desc}: {bar} {percentage:3.0f}% | {n_fmt}/{total_fmt}',
                     ncols=100, leave=False) as pbar:

                pbar.set_description(f"  {pasos[0]}")
                img_rgb, metadatos_carga = self.cargador.cargar_imagen(ruta_imagen)
                pbar.update(1)

                if img_rgb is None:
                    raise ValueError("No se pudo cargar la imagen")

                pbar.set_description(f"  {pasos[1]}")
                img_rgb_procesada = self._redimensionar_si_necesario(img_rgb)
                pbar.update(1)

                resultado = {
                    'metadatos_archivo': {
                        'nombre_archivo': ruta_imagen.name,
                        'extension': ruta_imagen.suffix.lower(),
                        **metadatos_carga
                    }
                }

                if self.config.calcular_exif:
                    pbar.set_description(f"  {pasos[2]}")
                    resultado['metadatos_exif'] = self.extractor_exif.extraer_exif(ruta_imagen)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_histogramas:
                    pbar.set_description(f"  {pasos[3]}")
                    resultado['estadisticas_color'] = self.analizador_color.analizar_color_completo(img_rgb_procesada)
                    pbar.update(1)

                    pbar.set_description(f"  {pasos[4]}")
                    resultado['histogramas'] = self.analizador_histogramas.analizar_histogramas(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(2)

                if self.config.calcular_texturas:
                    pbar.set_description(f"  {pasos[5]}")
                    resultado['texturas'] = self.analizador_textura.analizar_textura_completa(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_bordes:
                    pbar.set_description(f"  {pasos[6]}")
                    resultado['bordes_caracteristicas'] = self.analizador_bordes.analizar_bordes_completo(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_calidad:
                    pbar.set_description(f"  {pasos[7]}")
                    resultado['calidad'] = self.analizador_calidad.analizar_calidad_completa(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_frecuencias:
                    pbar.set_description(f"  {pasos[8]}")
                    resultado['analisis_frecuencia'] = self.analizador_frecuencia.analizar_frecuencias_completo(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_saliencia:
                    pbar.set_description(f"  {pasos[9]}")
                    resultado['saliencia_visual'] = self.analizador_saliencia.analizar_saliencia_completa(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_colores_dominantes:
                    pbar.set_description(f"  {pasos[10]}")
                    resultado['colores_dominantes'] = self.extractor_colores.extraer_colores_dominantes(img_rgb_procesada)
                    pbar.update(1)
                else:
                    pbar.update(1)

                if self.config.calcular_hashes:
                    pbar.set_description(f"  {pasos[11]}")
                    resultado['hashes_perceptuales'] = self.calculador_hashes.calcular_hashes(Image.fromarray(img_rgb_procesada))
                    pbar.update(1)
                else:
                    pbar.update(1)

                pbar.set_description(f"  {pasos[12]}")

                tiempo_procesamiento = time.time() - tiempo_inicio
                resultado['procesamiento'] = {
                    'timestamp': datetime.now().isoformat(),
                    'tiempo_segundos': round(tiempo_procesamiento, 3)
                }

                if self.config.guardar_json_individual:
                    self._guardar_json_individual(resultado)

                pbar.update(1)

                self.estadisticas_procesamiento['exitosas'] += 1
                self.estadisticas_procesamiento['tiempo_total'] += tiempo_procesamiento

                self.logger.info(f"Procesado exitosamente en {tiempo_procesamiento:.2f}s")

            return resultado

        except Exception as e:
            self.logger.error(f"Error procesando {ruta_imagen.name}: {str(e)}")
            import traceback
            self.logger.error(traceback.format_exc())
            self.estadisticas_procesamiento['errores'] += 1
            return None

    def _redimensionar_si_necesario(self, img: np.ndarray) -> np.ndarray:
        """Redimensiona imagen si excede dimensión máxima."""
        h, w = img.shape[:2]
        max_dim = max(h, w)

        if max_dim > self.config.max_dimension_procesamiento:
            escala = self.config.max_dimension_procesamiento / max_dim
            nuevo_w = int(w * escala)
            nuevo_h = int(h * escala)

            self.logger.info(f"Redimensionando de {w}x{h} a {nuevo_w}x{nuevo_h}")
            return cv2.resize(img, (nuevo_w, nuevo_h), interpolation=cv2.INTER_AREA)

        return img

    def _guardar_json_individual(self, resultado: Dict[str, Any]) -> None:
        """Guarda JSON individual para una imagen."""
        nombre_archivo = resultado['metadatos_archivo']['nombre_archivo']
        nombre_base = Path(nombre_archivo).stem
        ruta_json = self.config.ruta_resultados / "json" / f"{nombre_base}_caracteristicas.json"

        try:
            with open(ruta_json, 'w', encoding='utf-8') as f:
                json.dump(
                    resultado, f,
                    indent=2,
                    ensure_ascii=False,
                    default=Utilidades.convertir_a_serializable
                )
            self.logger.debug(f"JSON individual guardado: {ruta_json.name}")
        except Exception as e:
            self.logger.error(f"Error guardando JSON individual: {str(e)}")

    def procesar_directorio(self) -> List[Dict[str, Any]]:
        """Procesa todas las imágenes JPEG en el directorio configurado."""
        archivos = []
        for ext in CargadorImagen.EXTENSIONES_VALIDAS:
            archivos.extend(self.config.ruta_imagenes.glob(f"*{ext}"))
            archivos.extend(self.config.ruta_imagenes.glob(f"*{ext.upper()}"))

        archivos = sorted(set(archivos))

        total = len(archivos)
        self.estadisticas_procesamiento['total'] = total

        if total == 0:
            self.logger.warning(f"No se encontraron imágenes JPEG en {self.config.ruta_imagenes}")
            return []

        self.logger.info(f"\nIniciando procesamiento de {total} imágenes")
        self.logger.info(f"Directorio: {self.config.ruta_imagenes}\n")

        print("\n" + "=" * 80)
        print(f"PROCESANDO {total} FOTOGRAFÍAS")
        print("=" * 80 + "\n")

        for ruta_imagen in tqdm(archivos, desc="Progreso general",
                                 unit="foto", ncols=100, colour='green'):
            resultado = self.procesar_imagen(ruta_imagen)

            if resultado:
                self.resultados.append(resultado)

        print()

        return self.resultados

    def guardar_consolidado(self) -> None:
        """Guarda JSON consolidado con todos los resultados."""
        ruta_json = self.config.ruta_resultados / "json" / "caracteristicas_consolidadas.json"

        try:
            datos_consolidados = {
                'metadata': {
                    'fecha_generacion': datetime.now().isoformat(),
                    'total_imagenes': len(self.resultados),
                    'configuracion': {
                        'ruta_imagenes': str(self.config.ruta_imagenes),
                        'max_dimension_procesamiento': self.config.max_dimension_procesamiento,
                        'caracteristicas_calculadas': {
                            'exif': self.config.calcular_exif,
                            'histogramas': self.config.calcular_histogramas,
                            'texturas': self.config.calcular_texturas,
                            'bordes': self.config.calcular_bordes,
                            'calidad': self.config.calcular_calidad,
                            'frecuencias': self.config.calcular_frecuencias,
                            'saliencia': self.config.calcular_saliencia,
                            'colores_dominantes': self.config.calcular_colores_dominantes,
                            'hashes': self.config.calcular_hashes,
                        }
                    }
                },
                'resultados': self.resultados
            }

            with open(ruta_json, 'w', encoding='utf-8') as f:
                json.dump(
                    datos_consolidados, f,
                    indent=2,
                    ensure_ascii=False,
                    default=Utilidades.convertir_a_serializable
                )

            self.logger.info(f"JSON consolidado guardado: {ruta_json}")
        except Exception as e:
            self.logger.error(f"Error guardando JSON consolidado: {str(e)}")

    def generar_reporte(self) -> Dict[str, Any]:
        """Genera reporte de procesamiento."""
        return {
            'total_imagenes': self.estadisticas_procesamiento['total'],
            'exitosas': self.estadisticas_procesamiento['exitosas'],
            'con_error': self.estadisticas_procesamiento['errores'],
            'tiempo_total_segundos': round(self.estadisticas_procesamiento['tiempo_total'], 2),
            'tiempo_promedio_segundos': round(
                self.estadisticas_procesamiento['tiempo_total'] /
                max(self.estadisticas_procesamiento['exitosas'], 1), 2
            )
        }

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main():
    """Función principal de ejecución del sistema."""
    print("\n" + "=" * 80)
    print("EXTRACTOR DE CARACTERÍSTICAS FOTOGRÁFICAS - VERSIÓN 2.0 COMPLETA")
    print("=" * 80 + "\n")

    if not GestorDependencias.instalar_todas():
        print("\nERROR: No se pudieron instalar todas las dependencias")
        return None

    print()

    if not montar_google_drive():
        print("Advertencia: No se pudo montar Google Drive")

    config = ConfiguracionExtraccion(
        ruta_base=Path("/content/drive/MyDrive/TFM/"),
        max_dimension_procesamiento=2048,

        calcular_exif=True,
        calcular_histogramas=True,
        calcular_texturas=True,
        calcular_bordes=True,
        calcular_calidad=True,
        calcular_frecuencias=True,
        calcular_saliencia=True,
        calcular_colores_dominantes=True,
        calcular_hashes=True,
        num_colores_paleta=5,

        guardar_json_individual=True,
        guardar_json_consolidado=True,
        nivel_log="INFO"
    )

    es_valida, mensaje_error = config.validar()
    if not es_valida:
        print(f"\nERROR: Configuración inválida")
        print(f"Motivo: {mensaje_error}")
        return None

    extractor = ExtractorCaracteristicasImagen(config)
    resultados = extractor.procesar_directorio()

    if resultados and config.guardar_json_consolidado:
        extractor.guardar_consolidado()

    reporte = extractor.generar_reporte()

    print("\n" + "=" * 80)
    print("RESUMEN DE PROCESAMIENTO")
    print("=" * 80)
    print(f"Total imágenes procesadas: {reporte['total_imagenes']}")
    print(f"  Exitosas:                {reporte['exitosas']}")
    print(f"  Con errores:             {reporte['con_error']}")
    print(f"\nTiempo total:              {reporte['tiempo_total_segundos']:.2f}s")
    print(f"Tiempo promedio:           {reporte['tiempo_promedio_segundos']:.2f}s por imagen")
    print(f"\nResultados guardados en:")
    print(f"  {config.ruta_resultados / 'json'}")
    print("=" * 80 + "\n")

    return resultados

In [ ]:
# =============================================================================
# EJECUCIÓN
# =============================================================================

if __name__ == "__main__":
    resultados = main()


EXTRACTOR DE CARACTERÍSTICAS FOTOGRÁFICAS - VERSIÓN 2.0 COMPLETA

INSTALANDO DEPENDENCIAS

Instalando piexif... OK

Instalando exifread... OK

Instalando imagehash... OK

Instalando scikit-image... OK

Instalando mahotas... OK

Instalando opencv-python... OK

Instalando tqdm... OK

Resumen: 7 exitosas, 0 fallidas

Montando Google Drive... Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OK


PROCESANDO 20 FOTOGRAFÍAS



Progreso general:   0%|                                                    | 0/20 [00:00<?, ?foto/s]
  _DSC0023.jpg:                                                                            0% | 0/13
  Cargando imagen: :                                                                       0% | 0/13
  Cargando imagen: : █████▏                                                                8% | 1/13
  Redimensionando: : █████▏                                                                8% | 1/13
  Redimensionando: : ██████████▎                                                          15% | 2/13
  Extrayendo EXIF: : ██████████▎                                                          15% | 2/13
  Analizando color: : ███████████████▏                                                    23% | 3/13
  Analizando color: : ████████████████████▎                                               31% | 4/13
  Calculando histogramas: : ██████████████████▍                                           3



RESUMEN DE PROCESAMIENTO
Total imágenes procesadas: 20
  Exitosas:                20
  Con errores:             0

Tiempo total:              3446.57s
Tiempo promedio:           172.33s por imagen

Resultados guardados en:
  /content/drive/MyDrive/TFM/1_Caracteristicas/json

